In [2]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

In [3]:
pd.set_option('display.max_info_columns', 200)

In [12]:
%%time
data_ac = pd.read_csv('../data/accepted_2007_to_2018Q4.csv', low_memory=False)
data_rej = pd.read_csv('../data/rejected_2007_to_2018Q4.csv', low_memory=False)

CPU times: user 16.4 s, sys: 2.53 s, total: 18.9 s
Wall time: 23.7 s


In [13]:
data_ac.to_pickle('../data/data_accepted.pkl')
data_rej.to_pickle('../data/data_rejected.pkl')

In [45]:
%%time
data_ac = pd.read_pickle('../data/data_accepted.pkl', index_col='id')
data_rej = pd.read_pickle('../data/data_rejected.pkl')

CPU times: user 1.73 s, sys: 566 ms, total: 2.29 s
Wall time: 2.32 s


In [14]:
data_ac.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data_rej.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,1000.0,2007-05-26,Wedding Covered but No Honeymoon,693.0,10%,481xx,NM,4 years,0.0
1,1000.0,2007-05-26,Consolidating Debt,703.0,10%,010xx,MA,< 1 year,0.0
2,11000.0,2007-05-27,Want to consolidate my debt,715.0,10%,212xx,MD,1 year,0.0
3,6000.0,2007-05-27,waksman,698.0,38.64%,017xx,MA,< 1 year,0.0
4,1500.0,2007-05-27,mdrigo,509.0,9.43%,209xx,MD,< 1 year,0.0


In [16]:
data_rej.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12409081 entries, 0 to 12409080
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount Requested      float64
 1   Application Date      object 
 2   Loan Title            object 
 3   Risk_Score            float64
 4   Debt-To-Income Ratio  object 
 5   Zip Code              object 
 6   State                 object 
 7   Employment Length     object 
 8   Policy Code           float64
dtypes: float64(3), object(6)
memory usage: 852.1+ MB


In [17]:
data_ac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 786068 entries, 0 to 786067
Data columns (total 151 columns):
 #    Column                                      Non-Null Count   Dtype  
---   ------                                      --------------   -----  
 0    id                                          786068 non-null  object 
 1    member_id                                   0 non-null       float64
 2    loan_amnt                                   786060 non-null  float64
 3    funded_amnt                                 786060 non-null  float64
 4    funded_amnt_inv                             786060 non-null  float64
 5    term                                        786060 non-null  object 
 6    int_rate                                    786060 non-null  float64
 7    installment                                 786060 non-null  float64
 8    grade                                       786060 non-null  object 
 9    sub_grade                                   786060 non-nu

In [18]:
data_ac.loan_status.unique()

array(['Fully Paid', 'Current', 'Charged Off', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', 'Default', nan],
      dtype=object)

In [36]:
data_ac.policy_code.unique()

array([ 1., nan])

### Вывод по данным
Есть множество пропусков в каждом из датасетов
Датасет с отказами содержит значительно меньше признаков, чем в одобренных и чтобы не терять эти признаки(особенно loan_status), я приняла решение обучить две модели для разных таргетов
Первая модель будет обучена на обьединении одобренных и реджектнутых заявок только с общими признаками, таргетом будет - одобрение/отказ
Вторая модель будет обучена только на одобренных заявках, таргетом будет - значение loan_status бинаризированный



In [31]:
data_ac["approved"] = 1
data_rej["approved"] = 0

accepted = data_ac
rejected = data_rej

In [20]:
rejected["dti"] = rejected["Debt-To-Income Ratio"].str.rstrip('%').astype(float)
rejected["loan_amnt"] = rejected["Amount Requested"]
rejected["emp_length"] = rejected["Employment Length"]  # преобразовать
rejected["addr_state"] = rejected["State"]
rejected['zip_code'] = rejected['Zip Code']
rejected['policy_code'] = rejected['Policy Code']
rejected['purpose'] = rejected['Zip Code']


In [21]:
common_features = [
    "loan_amnt",       # Amount Requested
    "dti",             # Debt-To-Income Ratio
    "emp_length",      # Employment Length
    "addr_state",      # State
    "purpose",         # Loan Title / purpose
    "policy_code",      # Policy Code
    'zip_code'
]

In [22]:
accepted = accepted[common_features + ["approved"]]
rejected = rejected[common_features + ["approved"]]

In [24]:
full_data = pd.concat([accepted, rejected], axis=0)


In [25]:
full_data.head()

,loan_amnt,dti,emp_length,addr_state,approved
0,3600.0,5.91,10+ years,PA,1
1,24700.0,16.06,10+ years,SD,1
2,20000.0,10.78,10+ years,IL,1
3,35000.0,17.06,10+ years,NJ,1
4,10400.0,25.37,3 years,PA,1


In [26]:
full_data.emp_length.unique()

array(['10+ years', '3 years', '4 years', '6 years', '1 year', '7 years',
       '8 years', '5 years', '2 years', '9 years', '< 1 year', nan],
      dtype=object)

In [29]:
data_ac['zip_code'].value_counts()

zip_code
945xx    8182
112xx    8123
750xx    8079
606xx    7330
300xx    7314
         ... 
709xx       1
849xx       1
854xx       1
520xx       1
696xx       1
Name: count, Length: 924, dtype: int64

In [30]:
data_rej['Zip Code'].value_counts()

Zip Code
112xx    122179
770xx    117155
300xx    115472
331xx    111065
606xx    105886
          ...  
818xx         4
529xx         4
579xx         3
699xx         3
09Oxx         1
Name: count, Length: 1001, dtype: int64

In [39]:
data_rej['Loan Title'].value_counts()

Loan Title
debt_consolidation              2969543
Debt consolidation              2507541
other                           1065438
Other                            984052
Credit card refinancing          916744
                                 ...   
help ive tried every thing            1
Debt Consolidation/auto loan          1
Start-up Debt Loan                    1
Help me get out of this mess          1
peer to peer loan                     1
Name: count, Length: 73918, dtype: int64